In [ ]:
!pip install selenium==4.1.5 webdriver_manager openpyxl pandas
#크롬만들기
#크롬만들기
# -*- coding: UTF-8 -*-
# from google.colab import drive
# drive.mount('/content/drive')
import datetime
import random
from random import *

keywords = []

while True:
    keyword = input("키워드를 입력하세요(0을 입력하면 그만입력) : ")
    if keyword == "0":
        break
    keywords.append(keyword)

max_page = int(input('몇페이지까지 보실건가요?(40개보기기준-1 = 40개, 5 = 200개 , 10=400개)_정수만가능 :'))



print('네이버쇼핑접속 & 판매자 링크 크롤링중')

import time
st = 0.2
df_list = [['상호명','스토어명','분류(공란)','홈페이지','제품명','이메일','사업자 등록번호','키워드','제품URL','고객센터']]


#------------------------------------인터넷

#jupyter notebook


from selenium.webdriver.common.keys import Keys
#실습에 필요한 라이브러리가 가져오기(추후 설명 예정)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
service = Service(executable_path=ChromeDriverManager().install()) #크롬드라이버 설치
driver = webdriver.Chrome(service=service)






#colab

# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# from selenium import webdriver
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')        # Head-less 설정
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome('chromedriver', options=options)
# from selenium.webdriver.common.keys import Keys

#------------------------------------인터넷


import pandas as pd
# driver=webdriver.Chrome('c:\chromedriver.exe')


for keyword in keywords:

    df_list = [['상호명','스토어명','분류(공란)','홈페이지','제품명','이메일','사업자 등록번호','키워드','제품URL','고객센터']]
    shop_list = []
    for num in range(max_page):
#         try:
        page_url = f'https://search.shopping.naver.com/search/all?origQuery={keyword}&pagingIndex={str(num+1)}&pagingSize=40&productSet=total&query={keyword}'
        driver.get(page_url)
        for a in range(20):
            time.sleep(0.2)
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            #URL찾기
        first_sels = driver.find_elements_by_class_name('basicList_list_basis__uNBZx > div > div')
        for first_sel in first_sels:
            try:
                product_name = first_sel.find_element_by_class_name('product_title__Mmw2K').text
                print(product_name)
                catagory_name = first_sel.find_element_by_class_name('product_depth__I4SqY').text
                print(catagory_name)

                # fake_url_sel = first_sel.find_element_by_class_name('basicList_mall_title__FDXX5')
                # print(fake_url_sel)

                # fake_url = fake_url_sel.get_attribute('innerHTML').split('href="')[1].split('"')[0]
                # print(fake_url)
                fake_url = first_sel.find_element(By.CLASS_NAME,'product_link__TrAac.linkAnchor').get_attribute('href')
                print(fake_url)


                #상품 url
                prod_url = first_sel.find_element(By.CLASS_NAME,'product_link__TrAac.linkAnchor').get_attribute('href')
                print(prod_url)

                shop_list.append([product_name,catagory_name,fake_url,prod_url])
            except:
                print("페이지오류")
#         except:
#             print('페이지오류')
#             print(num+1)

    #fakeurl 들어가서
    for shop_unit in shop_list[:]:
        try:
            st = uniform(1,5)
            time.sleep(st)
            driver.get(shop_unit[2])
            st = uniform(1,5)
            time.sleep(st)
            #1스마트스토어인지 체크
            product_url = driver.current_url
            product_url
            seller_id = product_url.split('.com/')[1].split('products')[0].replace('/','')
            if "=" in seller_id:
                print('스마트스토어아님')
                continue

            #2스마트 스토어일 경우 코드를 돌려서 상호명/스토어명/분류/현재URL/제품명/이메일 을 취합
            profile_url = f"https://smartstore.naver.com/{seller_id}/profile"
            driver.get(profile_url)
            profile_main_sel = driver.find_element_by_class_name('_1Ve96aZgaO')
            profile_sels = profile_main_sel.find_elements_by_class_name('_2GWMVHvyCw')
            store_name = driver.find_element_by_class_name('name').text
            print('스토어명:',store_name)
            for profile_set in profile_sels:
                check = profile_set.find_elements_by_tag_name('div')[0].text
                if "상호명" in check:
            #         print(profile_set.find_elements_by_tag_name('div')[0].text)
                    sangho_name = profile_set.find_elements_by_tag_name('div')[1].text
                if "사업자등록번호" in check:
            #         print(profile_set.find_elements_by_tag_name('div')[0].text)
                    biz_num = profile_set.find_elements_by_tag_name('div')[1].text
                if 'e-mail'in check:
            #         print(profile_set.find_elements_by_tag_name('div')[0].text)
                    email = profile_set.find_elements_by_tag_name('div')[1].text
                if '고객센터'in check:
            #         print(profile_set.find_elements_by_tag_name('div')[0].text)
                    aspn = profile_set.find_elements_by_tag_name('div')[1].text.split('\n')[0].replace('인증','')
            print(sangho_name,store_name,shop_unit[1],product_url,shop_unit[0],email,biz_num,keyword,prod_url,aspn)
            df_list.append([sangho_name,store_name,shop_unit[1],profile_url.replace('/profile',''),shop_unit[0],email,biz_num,keyword,shop_unit[3],aspn])

            df_list.to_pickle("df.pkl")
            df_temp = pd.DataFrame(df_list)
            df_temp2 = df_temp.rename(columns=df_temp.iloc[0])
            df_temp3 = df_temp2.drop(df_temp2.index[0])
            df_temp3 = df_temp3[['키워드','상호명', '스토어명', '분류(공란)','제품URL', '홈페이지', '제품명', '이메일', '사업자 등록번호',
                    '고객센터']]
            df_temp3 = df_temp3.drop_duplicates(['스토어명'])
            # df3.to_excel(f'/content/drive/My Drive/{keyword}_{now}_조사.xlsx')
            df_temp3.to_excel(f'{keyword}_조사(임시).xlsx')





        except:
            print('next')

    df = pd.DataFrame(df_list)
    df2 = df.rename(columns=df.iloc[0])
    df3 = df2.drop(df2.index[0])
    df3 = df3[['키워드','상호명', '스토어명', '분류(공란)','제품URL', '홈페이지', '제품명', '이메일', '사업자 등록번호',
            '고객센터']]
    import datetime
    now = datetime.datetime.now()
    now = str(now).replace(":","_")[:-7]

    df3 = df3.drop_duplicates(['스토어명'])


    # df3.to_excel(f'/content/drive/My Drive/{keyword}_{now}_조사.xlsx')
    df3.to_excel(f'{keyword}_{now}_조사.xlsx')

    #전체반복


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


키워드를 입력하세요(0을 입력하면 그만입력) :  애견간식
키워드를 입력하세요(0을 입력하면 그만입력) :  고양이사료
키워드를 입력하세요(0을 입력하면 그만입력) :  0
몇페이지까지 보실건가요?(40개보기기준-1 = 40개, 5 = 200개 , 10=400개)_정수만가능 : 5


네이버쇼핑접속 & 판매자 링크 크롤링중


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.30M/6.30M [00:00<00:00, 58.6MB/s]
C:\Users\MK-Desktop\AppData\Local\Temp\ipykernel_16360\548768149.py:79: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
C:\Users\MK-Desktop\AppData\Local\Temp\ipykernel_16360\548768149.py:81: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  first_sels = driver.find_elements_by_class_name('basicList_list_basis__uNBZx > div > div')
C:\Users\MK-Desktop\AppData\Local\Temp\ipykernel_16360\548768149.py:84: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  product_name = first_sel.find_element_by_class_name('product_title__Mmw2K').text
C:\Users\MK-Desktop\AppData\Local\Temp\ipykernel_1

페이지오류
페이지오류
페이지오류
강아지 수제 간식 보양식 오리목뼈 100g+100g 대형견 반려견
생활/건강반려동물강아지 간식수제간식
https://cr.shopping.naver.com/adcr.nhn?x=wCKKuKOG9vMhneZSJK%2F%2F2f%2F%2F%2Fw%3D%3Ds2ogMyq6wt80knMktVdGVVmL%2B3m9dECHZ0W9bmyBAoL9wQXp09GSH%2F1C1HeQO8jgb%2FjkpS0G4xku9A9ZMFhnngfMkKj7yKEoLt55U0V2Q47QMrZTZoXdnufy1s%2FmdXgzs3h6AHb%2Bb2UDB2xTZKYy8njaSZny%2B1MgSfJG5ocTRQNj1rIvubP2%2F%2BNF2sTRY2hEViuEPVuqeg7X%2FKNUd0O6ssa3VeKW%2BkufeyUpuBIGiCLPL%2Fowdnq7%2Fmfq67OVRzok0S5SJNLgIdS1XFCtZ5GETGAgJbuuSxpxK1%2BpxGJsfUWBkMFEmGFYcGFPGevmjJvs2RQU%2FV78J5PLhMLEQRL3CfgYeoGRB5dauDbF7MLjppPHdcKGS8K3v9HOi3wvF0Lzm4Tx%2F2%2FVCjV0ASbrNHGkimfly35KMyPkYptbd%2B%2FpdrmEDvjg7wCd5C9HP6b0PGW5v23yrRFGhrTviqHxQ4wfFucK7F%2FL4JkPgXJ9S8qSODB3OMYIHOJkPe07YcDbaV1vHRk5K0RwCYnJYQypPhK5XtymEzheSrB6vooBgHNaQHmceua%2F%2FUh4wLc2IXb0DA1pKtirh0twDt2O8zSNuEtcBw5VDZvUsyAQfORRu13FJawNoaKZHilHbuauHUigNM5KA&nvMid=82440011828&catId=50006639
https://cr.shopping.naver.com/adcr.nhn?x=wCKKuKOG9vMhneZSJK%2F%2F2f%2F%2F%2Fw%3D%3Ds2ogMyq6wt80knMktVdGVVmL%2B3m9dECHZ0W9bmyB

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0065A813+48355]
	(No symbol) [0x005EC4B1]
	(No symbol) [0x004F5358]
	(No symbol) [0x004DD293]
	(No symbol) [0x0053E37B]
	(No symbol) [0x0054C473]
	(No symbol) [0x0053A536]
	(No symbol) [0x005182DC]
	(No symbol) [0x005193DD]
	GetHandleVerifier [0x008BAABD+2539405]
	GetHandleVerifier [0x008FA78F+2800735]
	GetHandleVerifier [0x008F456C+2775612]
	GetHandleVerifier [0x006E51E0+616112]
	(No symbol) [0x005F5F8C]
	(No symbol) [0x005F2328]
	(No symbol) [0x005F240B]
	(No symbol) [0x005E4FF7]
	BaseThreadInitThunk [0x767200C9+25]
	RtlGetAppContainerNamedObjectPath [0x770C7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x770C7B1E+238]
